In [17]:
import numpy as np
import pandas as pd

In [40]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_hero = pd.read_json('data/heros.json', orient='records')

In [34]:
df_hero = df_hero['heroes']

In [35]:
df_hero

0      {'name': 'antimage', 'id': 1, 'localized_name'...
1      {'name': 'axe', 'id': 2, 'localized_name': 'Axe'}
2      {'name': 'bane', 'id': 3, 'localized_name': 'B...
3      {'name': 'bloodseeker', 'id': 4, 'localized_na...
4      {'name': 'crystal_maiden', 'id': 5, 'localized...
                             ...                        
107    {'name': 'phoenix', 'id': 110, 'localized_name...
108    {'name': 'techies', 'id': 105, 'localized_name...
109    {'name': 'oracle', 'id': 111, 'localized_name'...
110    {'name': 'winter_wyvern', 'id': 112, 'localize...
111    {'name': 'arc_warden', 'id': 113, 'localized_n...
Name: heroes, Length: 112, dtype: object

In [36]:
df_train

,cluster_id,game_mode,game_type,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,...,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111,hero_112,hero_113,outcome
0,223,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
1,152,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
2,131,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
3,154,2,2,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,team_1
4,171,2,3,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,154,2,3,1,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
92646,154,2,2,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92647,111,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92648,185,2,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,team_2


In [48]:
def dataFilter(data):
    
    data = df_train.drop(["cluster_id", "game_mode", "game_type"], axis=1)
    features = data.drop("outcome", axis=1)
    classes = data.outcome
    return features, classes

In [49]:
df_train_features, classes = dataFilter(df_train)


In [50]:
df_train_features

,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,hero_10,...,hero_104,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111,hero_112,hero_113
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,-1,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,-1,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,1,0,0,-1,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
92646,0,0,0,0,-1,0,0,-1,0,0,...,1,0,0,0,0,0,0,0,0,0
92647,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
92648,0,0,0,0,0,1,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
classses

NameError: name 'classses' is not defined

In [ ]:
#return last pick given (1,2,3,4,5,6,7,8,9) [1,2,3,4,?] [5,6,7,8,9]
#for i in range(1,114) try everything but cannot pick currently selected heroes, and find best